In [12]:
import pandas as pd 


file = r'C:\Users\Lenovo\Desktop\urdu_sarcastic_dataset.csv'
df = pd.read_csv(file)


with open(r'C:\Users\Lenovo\Desktop\urdu_stopwords.txt', 'r', encoding='utf-8') as f:
    stopWords = f.read().splitlines()  

# List of sentiment-related words have impact on meaning
sentiment_words = {
    'نہیں', 'نہ', 'کچھ نہیں', 'کبھی نہیں', 'شاید', 'ضروری', 'بالکل', 'کافی',
    'لیکن', 'کیوں', 'کبھی', 'کبھی کبھار'
}

sample = df.head(15).copy()

def remove_stopWords(text):
    cleaned_text = [] 
    words = text.split() 
    
    for word in words:
        if word in stopWords and word not in sentiment_words:
            continue  # Skip the word if it's a stop word and not a sentiment word
        else:
            cleaned_text.append(word)  
    
    return ' '.join(cleaned_text) 


sample['cleaned_text'] = sample['urdu_text'].apply(remove_stopWords)

print(sample[['urdu_text', 'cleaned_text']])


                                            urdu_text  \
0   🤣😂😂 ہو لینے دے میری شادی فسادن ٹھیک ہے کوجی نہ...   
1   چل مہمانوں میں کھانا سرو کر چڑیل چاچی نوں دسدی...   
2   کامران خان آپکی دن بھریہ زمہ داری لگائی گئی اپ...   
3                                        نہیں پائین 😎   
4    `` مراد علی شاہ کے بھیس میں ڈی جی آئی ایس آئی...   
5         قابل اعتبار ہی اکثر قاتل اعتبار ہوتے ہیں 💔🔥   
6       انساں کو تھکا دیتا ہے سوچوں کا سفر بھی ... 🍁🥀   
7                               حامد میر صاحب ویلڈن👏😊   
8   یار وچارہ ویلا ہوندا ہے اس آرے لگا ہویا ہے😂😂 ت...   
9            یہ سمجھتے ہیں سارا پاکستان بیوقوف ھے 😂😂😂   
10                      تسی لڑاںٔی کروانی ساڈی کی 😂😂😂   
11                       پائن دوبارہ فالو کرئیے..؟؟😆🙄   
12  کتنی مہنگائی ہے الو دوسو روپے درجن کدو 80روپے ...   
13   😍عشق جب تم کو راس آۓ گا 💔زخم کھاٶ گے 😊مُسکراٶ گے   
14                                چونا ایسا ہی ہوتا 😂   

                                         cleaned_text  
0       🤣😂😂 لینے شادی فسادن ٹھی

There are multiple words that have impact on the meaning as I don't able to find a proper way or logic that how can we preserve them in cleaning stemming and lemmatization But I tried my best 

In [36]:
import re
import pandas as pd


file = r'C:\Users\Lenovo\Desktop\urdu_sarcastic_dataset.csv'
df = pd.read_csv(file, usecols=['urdu_text', 'is_sarcastic'])

with open(r'C:\Users\Lenovo\Desktop\urdu_stopwords.txt', 'r', encoding='utf-8') as f:
    stopWords = f.read().splitlines()

emoji_sentiment_urdu = {
    '😊': 'خوش',
    '😢': 'اداس',
    '😂': 'خوش',
    '🤣': 'خوش',
    '😐': 'غیر جانبدار',
    '😡': 'غصہ',
    '😠': 'غصہ',
    '😍': 'محبت',
    '😞': 'مایوس',
    '😭': 'اداس',
    '👍': 'اچھا',
    '👎': 'برا',
}
sentiment_words = {
    'نہیں', 'نہ', 'کچھ نہیں', 'کبھی نہیں', 'شاید', 'ضروری', 'بالکل', 'کافی',
    'لیکن', 'کیوں', 'کبھی', 'کبھی کبھار'
}
def clean_text(text):
    if not isinstance(text, str):  # non-string entries otherwise cause float error
        return text
    
    #tried to convert emojis to their meaning
    #for emoji, sentiment in emoji_sentiment_urdu.items():
    #    text = text.replace(emoji, sentiment)
    # URLs
    text = re.sub(r'http\S+', '', text)
    
    # hashtags
    text = re.sub(r'#\S+', '', text)
    
    # punctuation (but keep exclamation marks and Urdu punctuation like '؟') as they impact the meaning
    text = re.sub(r'[^\w\s!؟]', '', text)

    
    cleaned_text = []
    words = text.split()
    for word in words:
        if word in stopWords and word not in sentiment_words:
            continue  
        cleaned_text.append(word)  
    
    return ' '.join(cleaned_text)

def filter_short_posts(text):
    if not isinstance(text,str):
        return False
    words = text.split()  
    if len(words) < 3:  
        return False
    
    return text 

df['urdu_text'] = df['urdu_text'].apply(clean_text)
df['urdu_text'] = df['urdu_text'].apply(filter_short_posts)
df_cleaned = df[df['urdu_text'] != False]

df_cleaned.to_csv(r'C:\Users\Lenovo\Desktop\clean_text.csv', index=False, encoding='utf-8-sig')

print("Text preprocessing complete and file saved as clean_text.csv!")


Text preprocessing complete and file saved as clean_text.csv!


In [46]:
import pandas as pd
import re

file = r'C:\Users\Lenovo\Desktop\clean_text.csv'
df = pd.read_csv(file)


def stem_word(word):
    # common suffixes upto my findings 
    suffixes = ['وں', 'یں', 'ی', 'ا', 'ے', 'وں', 'ی', 'ہ']
    
    # plural forms (e.g., اچھے → اچھا)
    if word.endswith('ے') or word.endswith('وں'):
        return word[:-1]
    
    # gender forms (e.g., اچھی → اچھا)
    if word.endswith('ی'):
        return word[:-1]
    
    # verb inflections (e.g., چل رہی → چل)
    if word.endswith('رہی') or word.endswith('رہے'):
        return word[:-3]
    
    # end with a specific suffix remove it
    for suffix in suffixes:
        if word.endswith(suffix):
            return word[:-len(suffix)]
    
    # If none return as it is
    return word


def stem_text(text):
    
    words = text.split()
    

    stemmed_words = []
    for word in words:
        stemmed_word = stem_word(word)
        stemmed_words.append(stemmed_word)
    
    
    return ' '.join(stemmed_words)


df['stemmed_text'] = df['urdu_text'].apply(stem_text)


output_file = r'C:\Users\Lenovo\Desktop\stemmed.csv'
df.to_csv(output_file, index=False, encoding='utf-8-sig')


print(df[['urdu_text', 'stemmed_text']].head(10))


                                           urdu_text  \
0               لینے شادی فسادن ٹھیک کوجی نہیں چاہیے   
1         چل مہمانوں کھانا سرو چڑیل چاچی نوں دسدی آں   
2  کامران خان آپکی دن بھریہ زمہ داری لگائی اپوزیش...   
3               مراد علی شاہ بھیس ڈی جی ایس حامد میر   
4                       قابل اعتبار اکثر قاتل اعتبار   
5                               انساں تھکا سوچوں سفر   
6                                     حامد میر ویلڈن   
7  یار وچارہ ویلا ہوندا آرے ہویا تسی تے پکے نجومی...   
8                      سمجھتے سارا پاکستان بیوقوف ھے   
9                              تسی لڑاںی کروانی ساڈی   

                                        stemmed_text  
0                     لین شاد فسادن ٹھیک کوج نہ چاہی  
1              چل مہمانو کھان سرو چڑیل چاچ نو دسد آں  
2  کامران خان آپک دن بھری زم دار لگائ اپوزیشن کرد...  
3                   مراد عل شا بھیس ڈ ج ایس حامد میر  
4                       قابل اعتبار اکثر قاتل اعتبار  
5                                 انساں تھک سوچو سفر 

If we do only lemmatization the data has a lot of change and there are number of words that losses their actual meaning like shaadi is converted into shhad and achaa is aachh that has no meaning so I tried apply this with combo of pos tagging and lemmatization this helps to preserve the acens of word upto some extent

In [28]:
import pandas as pd
#these are some words that I can find while exploring data that should be preserved these cna be a large number 
#but as I don't find a way to automate this so using just small dictionary
lemmatization_dict = {
    'شادی': 'شادی',  
    'شاد': 'شادی',   
    'چل رہی': 'چل',  
    'لینے': 'لینا',  
    'خوبصورت': 'خوبصورتی',  
}


def pos_tagging(word):
    if word.endswith('ی') or word.endswith('ا'):
        return 'adjective'  # Examp words ending in ی or ا are adjectives
    elif word.endswith('ے') or word.endswith('ون'):
        return 'plural'
    else:
        return 'verb'  


def lemmatize_word(word):
    # if already lemmatized
    if word in lemmatization_dict:
        return lemmatization_dict[word]
    
    pos = pos_tagging(word)
    
    if pos == 'plural':
        # plural cases, e.g., 'لینے' -> 'لینا' 
        if word.endswith('ے'):
            return word[:-1] + 'ا'
    
    # If adjective, not to modify
    if pos == 'adjective':
        return word
    
    # Return the word unchanged if no rules apply
    return word


def lemmatize_text(text):
    words = text.split()
    lemmatized_words = [lemmatize_word(word) for word in words]
    return ' '.join(lemmatized_words)


file = r'C:\Users\Lenovo\Desktop\stemmed.csv'
df = pd.read_csv(file)


df['lemmatized_text'] = df['stemmed_text'].apply(lemmatize_text)


output_file=r'C:\Users\Lenovo\Desktop\lemmatized.csv'
df.to_csv(output_file, index=False, encoding='utf-8-sig')


In [52]:
import pandas as pd

def tokenize_text(text):
    tokens = text.split()
    return tokens

file=r'C:\Users\Lenovo\Desktop\lemmatized.csv'
df = pd.read_csv(file)


df['tokenized_text'] = df['lemmatized_text'].apply(tokenize_text)

output_file=r'C:\Users\Lenovo\Desktop\tokenized.csv'
df.to_csv(output_file, index=False, encoding='utf-8-sig')


print(df[['lemmatized_text', 'tokenized_text']].head())


                                     lemmatized_text  \
0                    لین شادی فسادن ٹھیک کوج نہ چاہی   
1              چل مہمانو کھان سرو چڑیل چاچ نو دسد آں   
2  کامران خان آپک دن بھری زم دار لگائ اپوزیشن کرد...   
3                   مراد عل شا بھیس ڈ ج ایس حامد میر   
4                       قابل اعتبار اکثر قاتل اعتبار   

                                      tokenized_text  
0            [لین, شادی, فسادن, ٹھیک, کوج, نہ, چاہی]  
1    [چل, مہمانو, کھان, سرو, چڑیل, چاچ, نو, دسد, آں]  
2  [کامران, خان, آپک, دن, بھری, زم, دار, لگائ, اپ...  
3         [مراد, عل, شا, بھیس, ڈ, ج, ایس, حامد, میر]  
4                 [قابل, اعتبار, اکثر, قاتل, اعتبار]  


In [54]:
import pandas as pd
import math
from collections import Counter


file = r'C:\Users\Lenovo\Desktop\lemmatized.csv'
df = pd.read_csv(file)

def compute_tf(text):
    words = text.split() 
    word_count = len(words)
    tf_dict = {}

    for word in words:
        if word in tf_dict:
            tf_dict[word] += 1  # Increment count if word already seen
        else:
            tf_dict[word] = 1  # Initialize count for new word

    # Normalize by dividing each count by the total number of words
    for word in tf_dict:
        tf_dict[word] = tf_dict[word] / word_count

    return tf_dict


def compute_idf(df_column):
    N = len(df_column) 
    idf_dict = {}  

    # through each document
    for text in df_column:
        words_in_doc = set(text.split())  

        for word in words_in_doc:
            if word in idf_dict:
                idf_dict[word] += 1  # Increment count if word already seen in another document
            else:
                idf_dict[word] = 1  # Initialize count for a new word

    
    for word in idf_dict:
        idf_dict[word] = math.log(N / idf_dict[word])

    return idf_dict

# Step 3: Calculate TF-IDF
def compute_tf_idf(tf_dict, idf_dict):
    tf_idf_dict = {}  
    for word, tf_value in tf_dict.items():
        if word in idf_dict:  # Only compute if the word has an IDF value
            tf_idf_dict[word] = tf_value * idf_dict[word]  
    return tf_idf_dict

tf_list = []
tf_idf_list = []


for i in range(len(df)):
    tf = compute_tf(df['lemmatized_text'][i]) 
    tf_list.append(tf)  


idf_dict = compute_idf(df['lemmatized_text'])

# Calculate TF-IDF for each row
for i in range(len(df)):
    tf_idf = compute_tf_idf(tf_list[i], idf_dict)  
    tf_idf_list.append(tf_idf)  


df['tf'] = tf_list  
df['tf_idf'] = tf_idf_list  
output_file=r'C:\Users\Lenovo\Desktop\tf-idf.csv'
df.to_csv(output_file, index=False, encoding='utf-8-sig')

print("TF-IDF process completed and saved to tfidf_lemmatized.csv.")


TF-IDF process completed and saved to tfidf_lemmatized.csv.


In [61]:
import pandas as pd


file = r'C:\Users\Lenovo\Desktop\tf-idf.csv'
df = pd.read_csv(file)
tf_idf_pairs = []

for i in range(len(df)):
    tf_idf_dict = eval(df['tf_idf'][i])  # Convert the string back into a dictionary from csv file
    for word, score in tf_idf_dict.items():
        tf_idf_pairs.append((word, score))  # word and its score as a tuple


sorted_tf_idf_pairs = sorted(tf_idf_pairs, key=lambda x: x[1], reverse=True)


top_10_words = sorted_tf_idf_pairs[:10]

top_10_df = pd.DataFrame(top_10_words, columns=['Word', 'TF-IDF Score'])

print(top_10_df)



   Word  TF-IDF Score
0   پاں      9.115425
1    وہ      7.356429
2  ترسو      6.539048
3     ہ      6.512735
4   قیم      5.614852
5  جتوئ      5.469255
6  جتوئ      5.469255
7   چوٹ      5.466090
8  کھاو      5.241775
9  آداب      5.241775


In [14]:
import pandas as pd

from gensim.models import Word2Vec


file_path = r'C:\Users\Lenovo\Desktop\tokenized.csv' 
df = pd.read_csv(file_path)


df['tokenized_text'] = df['tokenized_text'].apply(eval)

tokenized_data = df['tokenized_text'].tolist()


model = Word2Vec(sentences=tokenized_data, vector_size=100, window=5, min_count=1, sg=0)


model.save("word2vec_urdu.model")


vocabulary = list(model.wv.index_to_key)
print("Vocabulary:", vocabulary)


similar_words = model.wv.most_similar("نسبت", topn=5)
print("Top 5 words most similar to 'نسبت':")
for word, score in similar_words:
    print(f"{word}: {score}")


Vocabulary: ['نہ', 'ن', 'ھ', 'پ', 'خان', 'الل', 'بات', 'ج', 'پاکستان', 'سندھ', 'بن', 'نواز', 'حکومت', 'عمران', 'آج', 'ساتھ', 'دیکھ', 'تنقید', 'لوگ', 'شریف', 'لیکن', 'پولیس', 'اچھ', 'اتن', 'کہت', 'سار', 'دل', 'بس', 'مل', 'بلاول', 'فوج', 'ت', 'مریم', 'خود', 'صفدر', 'پت', 'کیو', 'کبھ', 'پور', 'وقت', 'دن', 'لگت', 'لوگو', 'ملک', 'کام', 'نام', 'بھائ', 'عزت', 'عوام', 'بند', 'مزاح', 'روپ', 'چل', 'ھو', 'بار', 'خوش', 'گھر', 'کم', 'من', 'سمجھ', 'ڈ', 'زندگ', 'و', 'محبت', 'جان', 'چیف', 'ب', 'ہوگ', 'فالو', 'لکھ', 'دنی', 'نان', 'سکت', 'ویس', 'کل', 'نکل', 'چور', 'ختم', 'پڑ', 'دوسر', 'سال', 'آرم', 'سر', 'آمین', 'لیگ', 'اسک', 'قوم', 'پاک', 'کہ', 'جلس', 'گل', 'یاد', 'کت', 'وزیراعظم', 'بچ', 'سن', 'کتن', 'انسان', 'زردار', 'دور', 'نظر', 'چاہت', 'آپک', 'گرفتار', 'د', 'نئ', 'ٹ', 'ہم', 'نیاز', 'مار', 'خیر', 'کیڑ', 'کراچ', 'چک', 'اغو', 'جواب', 'کیپٹن', 'لین', 'عورت', 'شروع', 'ہمیش', 'باق', 'سوچ', 'عل', 'فرمائ', 'میاں', 'مر', 'بیٹھ', 'اپوزیشن', 'سیاست', 'مطلب', '؟', 'ہاتھ', 'بھٹو', 'انک', 'ایم', 'ٹویٹ', 'چیز', '

In [20]:
import pandas as pd
from collections import Counter

df = pd.read_csv(r'C:\Users\Lenovo\Desktop\tokenized.csv', encoding='utf-8-sig')

def create_ngrams(tokens, n):
    ngrams = []
    for i in range(len(tokens) - n + 1):
        ngrams.append(tuple(tokens[i:i+n]))
    return ngrams

unigram_counter = Counter()
bigram_counter = Counter()
trigram_counter = Counter()


for doc in df['tokenized_text']:
    tokens = eval(doc)  
    
    for token in tokens:
        unigram_counter.update([token])  

    bigrams = create_ngrams(tokens, 2)
    bigram_counter.update(bigrams)  
    
    trigrams = create_ngrams(tokens, 3)
    trigram_counter.update(trigrams)  


top_10_bigrams = bigram_counter.most_common(10)
top_10_trigrams = trigram_counter.most_common(10)


print("Top 10 Bigrams with Frequencies:")
for bigram, freq in top_10_bigrams:
    print(f"{bigram}: {freq}")

print("\nTop 10 Trigrams with Frequencies:")
for trigram, freq in top_10_trigrams:
    print(f"{trigram}: {freq}")


Top 10 Bigrams with Frequencies:
('عمران', 'خان'): 502
('نواز', 'شریف'): 446
('سندھ', 'پولیس'): 299
('آرم', 'چیف'): 224
('جزاک', 'الل'): 198
('کیپٹن', 'صفدر'): 177
('ن', 'لیگ'): 161
('مریم', 'نواز'): 159
('صل', 'الل'): 150
('پت', 'نہ'): 144

Top 10 Trigrams with Frequencies:
('صل', 'الل', 'علی'): 138
('جزاک', 'الل', 'خیر'): 99
('الل', 'علی', 'وآل'): 87
('علی', 'وآل', 'وسلم'): 87
('پ', 'ڈ', 'ایم'): 86
('فالو', 'فالو', 'بیک'): 71
('استغفر', 'الله', 'واتوب'): 53
('عط', 'فرمائ', 'آمین'): 52
('کیپٹن', 'صفدر', 'گرفتار'): 52
('الله', 'واتوب', 'اليه'): 51


Here we applied the NB classifier on base of tf-idf value, 
The reason for using Multinomial NB is nothing but a Guassian NB classifier 
which is used when data can be easily converted into counts like word counts
As we are applying it to tf-idf that is some how telling about the frequency of 
data.
https://stats.stackexchange.com/questions/33185/difference-between-naive-bayes-multinomial-naive-bayes 

In [27]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import ast

file_path = r'C:\Users\Lenovo\Desktop\tf-idf.csv'
data = pd.read_csv(file_path)

# vocabulary (unique words in all TF-IDF dictionaries)
vocab = set()
for i in range(len(data)):
    tfidf_dict = eval(data['tf_idf'][i])  
    for word in tfidf_dict:
        vocab.add(word)  

vocab = list(vocab)  


X = []
for i in range(len(data)):
    tfidf_dict = eval(data['tf_idf'][i])  # Convert string to dictionary
    vector = []  
    for word in vocab:  
        if word in tfidf_dict:  
            vector.append(tfidf_dict[word])  
        else:
            vector.append(0)  
    X.append(vector)

y = data['is_sarcastic']  


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


nb_model = MultinomialNB()
nb_model.fit(X_train, y_train)

y_pred = nb_model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("F1 Score:", f1_score(y_test, y_pred))


Accuracy: 0.7776250687190764
Precision: 0.746160409556314
Recall: 0.890983188996434
F1 Score: 0.8121662410030184


As it is applied to tf-idf which have some issues due to stemming and lemmatization
So just for sake of experiment I have applied this directly to cleaned text

In [30]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Load dataset
df = pd.read_csv('C:\\Users\\Lenovo\\Desktop\\clean_text.csv')

# Features and labels
X = df['urdu_text']
y = df['is_sarcastic']

# TF-IDF Vectorizer
tfidf = TfidfVectorizer()
X_tfidf = tfidf.fit_transform(X)

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

# Naive Bayes Classifier
nb = MultinomialNB()
nb.fit(X_train, y_train)

# Predictions
y_pred = nb.predict(X_test)

# Evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-Score: {f1}")


Accuracy: 0.7652556349642661
Precision: 0.724949290060852
Recall: 0.9103413143148242
F1-Score: 0.8071364046973803


### Natural Language Processing Assignment 1
### Shaheer Ahmad 21F9097 (9A)
**As I find manually some issues in stemming and lemmatization. SO I applied this directly to clean text and compare the results

### Detailed Findings: Stemming, Lemmatization, and TF-IDF vs. Direct Cleaned Text

We applied Naive Bayes classification to both:
1. **TF-IDF-based approach (using lemmatized text)**
2. **Direct application on cleaned text** (without stemming/lemmatization).

Here’s an analysis of the results:

#### Results Overview:
- **TF-IDF Approach**:
  - **Accuracy**: 77.76%
  - **Precision**: 74.62%
  - **Recall**: 89.10%
  - **F1 Score**: 81.21%

- **Direct Cleaned Text**:
  - **Accuracy**: 76.52%
  - **Precision**: 72.49%
  - **Recall**: 91.03%
  - **F1 Score**: 80.71%

---

### 1. **Accuracy**:
- **TF-IDF** has slightly **higher accuracy (77.76%)** compared to the **direct cleaned text (76.52%)**.
- This indicates that **lemmatization and stemming combined with TF-IDF capture slightly better overall performance** in terms of correct predictions.
  
### 2. **Precision**:
- Precision is **higher for the TF-IDF approach (74.62%)** compared to **direct cleaned text (72.49%)**.
- **Precision** measures the model's ability to correctly identify sarcastic comments without false positives. The **TF-IDF model is more precise** when predicting sarcasm.
  
### 3. **Recall**:
- Interestingly, **Recall is higher for direct cleaned text (91.03%)** vs. the **TF-IDF approach (89.10%)**.
- Recall measures how well the model can detect all sarcastic comments. **Direct cleaned text** is better at finding true sarcastic comments but may have higher false positives.

### 4. **F1-Score**:
- The **F1-Score** is a balance of Precision and Recall. It’s slightly higher for **TF-IDF (81.21%)** compared to **direct cleaned text (80.71%)**.
- This shows that the overall balance between Precision and Recall is better with the **TF-IDF-based approach**.

---

### Key Insights:
- **TF-IDF with Lemmatization and Stemming** shows better performance in **accuracy, precision, and F1 score**. This indicates that despite some loss of information during stemming and lemmatization, the **model benefits from the generalization introduced by reducing word forms**.
  
- **Direct cleaned text has better recall** but lower precision, meaning it detects sarcastic comments better but introduces more false positives, which can be problematic in real-world scenarios where false positives might mislead decisions.

---

### Is my TF-IDF with Stemming and Lemmatization logic used above Defensible?
**Yes, it is defensible** for several reasons:
1. **Slightly higher accuracy and precision** indicate that the model is more reliable in detecting sarcasm with fewer false positives.
2. The **F1 score**, a balance of Precision and Recall, is marginally better with **TF-IDF**, supporting the idea that stemming and lemmatization help generalize the text better.
3. Although **Recall is higher** for direct cleaned text, it comes at the cost of **lower precision**, meaning the model might overfit or predict more sarcastic comments than necessary.

### Impact of Stemming and Lemmatization:
- **Pros**:
  - Helps reduce dimensionality, especially in Urdu, where multiple word forms exist due to grammatical variations.
  - Improves generalization by treating different forms of the same word as a single feature (e.g., stemming “لینا” and “لیتا” to a root form).
  - Slightly improves **accuracy and precision**, which can be valuable for detecting sarcasm with fewer errors.

- **Cons**:
  - Stemming and Lemmatization might **remove important nuances** in sarcastic language. This loss of fine-grained data can be seen in the slightly lower **Recall**, meaning it sometimes misses subtle sarcastic comments.

---

### Conclusion:
- **TF-IDF with Stemming and Lemmatization is a strong approach** overall, especially in terms of balancing precision and recall. While there is some loss of recall, the higher precision and balanced F1 score make it more robust for sarcasm detection.
So if we handle the exceptions and have proper vocabulary of urdu available the results can be improved 